<center>

| Nazwa przedmiotu                          | Dataset                                |
| ----------------------------------------- | -------------------------------------- |
| Projekt zespołowy - sztuczna inteligencja | laptop_prices_dataset                  |
|                                           |                                        |
| Piotr                                     | Goraj                                  |
| Numer albumu                              | 55529                                  |
|                                           |                                        |
| Bartosz                                   | Kiałka                                 |
| Numer albumu                              | 55528                                  |
|                                           |                                        |
| Data oddania sprawozdania                 | 2024.11.16                             |
| Kierunek                                  | Informatyka, II stopnia P, Stacjonarne |

</center>


# 1. Instalacja i import potrzebnych bibliotek

In [1]:
# !pip install pandas
# !pip install scikit-learn
# !pip install numpy

In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from pandas import DataFrame

# 2. Odczyt danych

In [3]:
import pandas as pd

# załadowanie pliku CSV
FILE_PATH = './laptop_prices_dataset.csv'
data = pd.read_csv(FILE_PATH)

In [4]:
# data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Company               1275 non-null   object 
 1   Product               1275 non-null   object 
 2   TypeName              1275 non-null   object 
 3   Inches                1275 non-null   float64
 4   Ram                   1275 non-null   int64  
 5   OS                    1275 non-null   object 
 6   Weight                1275 non-null   float64
 7   Price_euros           1275 non-null   float64
 8   Screen                1275 non-null   object 
 9   ScreenW               1275 non-null   int64  
 10  ScreenH               1275 non-null   int64  
 11  Touchscreen           1275 non-null   object 
 12  IPSpanel              1275 non-null   object 
 13  RetinaDisplay         1275 non-null   object 
 14  CPU_company           1275 non-null   object 
 15  CPU_freq             

In [5]:
# pierwsze trzy rekordy
data.head(3)

,Company,Product,TypeName,Inches,Ram,OS,Weight,Price_euros,Screen,ScreenW,...,RetinaDisplay,CPU_company,CPU_freq,CPU_model,PrimaryStorage,SecondaryStorage,PrimaryStorageType,SecondaryStorageType,GPU_company,GPU_model
0,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1339.69,Standard,2560,...,Yes,Intel,2.3,Core i5,128,0,SSD,No,Intel,Iris Plus Graphics 640
1,Apple,Macbook Air,Ultrabook,13.3,8,macOS,1.34,898.94,Standard,1440,...,No,Intel,1.8,Core i5,128,0,Flash Storage,No,Intel,HD Graphics 6000
2,HP,250 G6,Notebook,15.6,8,No OS,1.86,575.00,Full HD,1920,...,No,Intel,2.5,Core i5 7200U,256,0,SSD,No,Intel,HD Graphics 620


In [6]:
# podsumowanie statystyczne
print(data.describe())

            Inches          Ram       Weight  Price_euros      ScreenW  \
count  1275.000000  1275.000000  1275.000000  1275.000000  1275.000000   
mean     15.022902     8.440784     2.040525  1134.969059  1900.043922   
std       1.429470     5.097809     0.669196   700.752504   493.346186   
min      10.100000     2.000000     0.690000   174.000000  1366.000000   
25%      14.000000     4.000000     1.500000   609.000000  1920.000000   
50%      15.600000     8.000000     2.040000   989.000000  1920.000000   
75%      15.600000     8.000000     2.310000  1496.500000  1920.000000   
max      18.400000    64.000000     4.700000  6099.000000  3840.000000   

           ScreenH     CPU_freq  PrimaryStorage  SecondaryStorage  
count  1275.000000  1275.000000     1275.000000       1275.000000  
mean   1073.904314     2.302980      444.517647        176.069020  
std     283.883940     0.503846      365.537726        415.960655  
min     768.000000     0.900000        8.000000          0.00

In [7]:
# czy występują wartości null
display(data.isnull().sum())

Company                 0
Product                 0
TypeName                0
Inches                  0
Ram                     0
OS                      0
Weight                  0
Price_euros             0
Screen                  0
ScreenW                 0
ScreenH                 0
Touchscreen             0
IPSpanel                0
RetinaDisplay           0
CPU_company             0
CPU_freq                0
CPU_model               0
PrimaryStorage          0
SecondaryStorage        0
PrimaryStorageType      0
SecondaryStorageType    0
GPU_company             0
GPU_model               0
dtype: int64

In [8]:
# Wybrane kolumny do modeli
selected_columns = [ 'Company','TypeName', 'Price_euros', 'Inches','Ram','OS','Touchscreen','CPU_freq','CPU_model','GPU_company','GPU_model']

# Wczytanie pliku CSV z wybranymi kolumnami
print("Wybrane kolumny CSV:")
df_selected_columns_csv = pd.read_csv(FILE_PATH, usecols=selected_columns)

# pierwsze 10 rekordów
display(df_selected_columns_csv.head(10))

Wybrane kolumny CSV:


,Company,TypeName,Inches,Ram,OS,Price_euros,Touchscreen,CPU_freq,CPU_model,GPU_company,GPU_model
0,Apple,Ultrabook,13.3,8,macOS,1339.69,No,2.3,Core i5,Intel,Iris Plus Graphics 640
1,Apple,Ultrabook,13.3,8,macOS,898.94,No,1.8,Core i5,Intel,HD Graphics 6000
2,HP,Notebook,15.6,8,No OS,575.00,No,2.5,Core i5 7200U,Intel,HD Graphics 620
3,Apple,Ultrabook,15.4,16,macOS,2537.45,No,2.7,Core i7,AMD,Radeon Pro 455
4,Apple,Ultrabook,13.3,8,macOS,1803.60,No,3.1,Core i5,Intel,Iris Plus Graphics 650
5,Acer,Notebook,15.6,4,Windows 10,400.00,No,3.0,A9-Series 9420,AMD,Radeon R5
6,Apple,Ultrabook,15.4,16,Mac OS X,2139.97,No,2.2,Core i7,Intel,Iris Pro Graphics
7,Apple,Ultrabook,13.3,8,macOS,1158.70,No,1.8,Core i5,Intel,HD Graphics 6000
8,Asus,Ultrabook,14.0,16,Windows 10,1495.00,No,1.8,Core i7 8550U,Nvidia,GeForce MX150
9,Acer,Ultrabook,14.0,8,Windows 10,770.00,No,1.6,Core i5 8250U,Intel,UHD Graphics 620


In [9]:
# Wydzielenie do osobnych plików zbiorów treningowego, walidacyjnego, oraz testowego.

# Podział na zbiór treningowy (70%) oraz zbiór tymczasowy (30% - do podziału na walidacyjny i testowy)
train_data, temp_data = train_test_split(df_selected_columns_csv, test_size=0.3, random_state=42)

# Podział zbioru tymczasowego na walidacyjny (15%) i testowy (15%)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# długość każdego zbioru
prime_len =  len(df_selected_columns_csv)
train_len = len(train_data)
val_len =  len(val_data)
test_len = len(test_data)

print("Długość zbioru:")
print("\tpierwotnego:", prime_len, "(100%)")
print("\ttreningowego:", train_len, "({:.2f}%)".format(train_len * 100 / prime_len))
print("\twalidacyjnego:", val_len, "({:.2f}%)".format(val_len * 100 / prime_len))
print("\ttestowego:", test_len, "({:.2f}%)".format(test_len * 100 / prime_len))

# pierwsze 5 rekordów każdego zbioru
display(train_data.head(5))
display(val_data.head(5))
display(test_data.head(5))

Długość zbioru:
	pierwotnego: 1275 (100%)
	treningowego: 892 (69.96%)
	walidacyjnego: 191 (14.98%)
	testowego: 192 (15.06%)


,Company,TypeName,Inches,Ram,OS,Price_euros,Touchscreen,CPU_freq,CPU_model,GPU_company,GPU_model
314,Asus,2 in 1 Convertible,11.6,2,Windows 10,275.00,Yes,1.1,Celeron Dual Core N3350,Intel,HD Graphics 500
1032,MSI,Gaming,17.3,8,Windows 10,1191.80,No,2.8,Core i7 7700HQ,Nvidia,GeForce GTX 1050
319,Acer,Notebook,11.6,4,Windows 10,485.00,No,1.6,Pentium Quad Core N3710,Intel,HD Graphics 405
1039,HP,Notebook,14.0,8,Windows 10,1010.51,No,2.5,Core i5 7200U,Intel,HD Graphics 620
966,Dell,Notebook,15.6,4,Windows 10,825.00,No,2.5,Core i5 7200U,Intel,HD Graphics 620


,Company,TypeName,Inches,Ram,OS,Price_euros,Touchscreen,CPU_freq,CPU_model,GPU_company,GPU_model
415,Dell,Notebook,15.6,4,Linux,598.9,No,2.5,Core i5 7200U,AMD,Radeon R5 M430
634,Asus,Notebook,15.6,8,Windows 10,399.0,No,1.1,Celeron Dual Core N3350,Intel,HD Graphics 500
1043,Lenovo,Notebook,15.6,8,Windows 10,1049.6,No,2.3,Core i5 6200U,Intel,HD Graphics 520
63,Asus,Notebook,14.0,8,Windows 10,941.0,No,1.6,Core i5 8250U,Intel,UHD Graphics 620
96,Dell,Notebook,15.6,8,Linux,599.9,No,2.7,Core i7 7500U,AMD,Radeon R5 M430


,Company,TypeName,Inches,Ram,OS,Price_euros,Touchscreen,CPU_freq,CPU_model,GPU_company,GPU_model
1050,Lenovo,Notebook,14.0,16,Windows 7,2620.00,No,2.6,Core i7 6600U,Intel,HD Graphics 520
701,Lenovo,Notebook,15.6,4,Windows 10,399.00,No,2.9,A9-Series 9420,AMD,Radeon 530
628,Asus,2 in 1 Convertible,13.3,6,Windows 10,639.01,Yes,2.5,Core i5 7200U,Intel,HD Graphics 620
467,Dell,Notebook,15.6,8,Windows 10,759.00,No,1.6,Core i5 8250U,AMD,Radeon 530
1006,HP,Notebook,14.0,4,Windows 10,1205.00,No,2.5,Core i5 7200U,Intel,HD Graphics 620


# 3. Przygotowanie danych do modelu

In [10]:
def replace_with_top_n(data: DataFrame, column: str, n: int, other_label: str ="Other"):
    """
    Zamienia wartości w kolumnie na n najczęściej występujących,
    a pozostałe wartości na 'Other'.
    
    Params:
    - column: Nazwa kolumny do przetworzenia.
    - n: Liczba najczęściej występujących wartości, które mają zostać zachowane.
    - other_label: Etykieta dla pozostałych wartości (domyślnie "Other").

    Returns:
    - DataFrame z przekształconą kolumną.
    """
    top_n = data[column].value_counts().nlargest(n).index
    data[column] = data[column].apply(lambda x: x if x in top_n else other_label)

def replace_with_log_transformation(data: DataFrame, column: str):
    """
    Zamienia wartości w kolumnie na wartości zlogarytmowane.

    Params:
    - column: Nazwa kolumny do przetworzenia.
    """
    data[column] = np.log1p(data[column])

def remove_model_number(data: DataFrame, column: str):
    """
    Usuwa końcowe cyfry (model) z nazwy, pozostawiając tylko część tekstową.
    
    Params:
    - text (str): Tekst do przetworzenia (np. "Core i5 7200U").
    
    Returns:
    - str: Przetworzony tekst bez końcowych cyfr (np. "Core i5").
    """
    data[column] = data[column].apply(lambda text: re.sub(r'\s+\d+\w*$', '', text))

def encode_with_dummies(data: DataFrame, columns: list):
    """
    Przekształca wybrane kolumny na zmienne zero-jedynkowe (one-hot encoding).
    
    Params:
    - data: DataFrame do przekształcenia.
    - columns: Lista kolumn kategorycznych do zakodowania.
    
    Returns:
    - DataFrame z zakodowanymi kolumnami.
    """
    data_encoded = pd.get_dummies(data, columns=columns, drop_first=True)  
    # drop_first zapobiega pułapce zmiennych fikcyjnych 
    # Usuwa pierwszą kolumnę dla każdej kategorii, 
    # aby uniknąć pułapki zmiennych fikcyjnych (tzw. dummy variable trap). 
    # Pomaga to w modelach regresji uniknąć współliniowości.
    return data_encoded


#### 3.1 Dane treningowe

In [11]:
# top 5 Company
replace_with_top_n(train_data, 'Company', 5)
replace_with_top_n(val_data, 'Company', 5)
replace_with_top_n(test_data, 'Company', 5)

# top 3 TypeName
replace_with_top_n(train_data, 'TypeName', 3)
replace_with_top_n(val_data, 'TypeName', 3)
replace_with_top_n(test_data, 'TypeName', 3)

# top 4 OS
replace_with_top_n(train_data, 'OS', 4)
replace_with_top_n(val_data, 'OS', 4)
replace_with_top_n(test_data, 'OS', 4)

# Top 5 CPU_model
remove_model_number(train_data, 'CPU_model')
remove_model_number(val_data, 'CPU_model')
remove_model_number(test_data, 'CPU_model')

replace_with_top_n(train_data, 'CPU_model', 5)
replace_with_top_n(val_data, 'CPU_model', 5)
replace_with_top_n(test_data, 'CPU_model', 5)

# Top 5 GPU_company
replace_with_top_n(train_data, 'GPU_company', 5)
replace_with_top_n(val_data, 'GPU_company', 5)
replace_with_top_n(test_data, 'GPU_company', 5)

# Top 5 GPU_model
remove_model_number(train_data, 'GPU_model')
remove_model_number(val_data, 'GPU_model')
remove_model_number(test_data, 'GPU_model')

replace_with_top_n(train_data, 'GPU_model', 5)
replace_with_top_n(val_data, 'GPU_model', 5)
replace_with_top_n(test_data, 'GPU_model', 5)

"""
"""

# Transformacja logarytmiczna Ram
replace_with_log_transformation(train_data, 'Ram')
replace_with_log_transformation(val_data, 'Ram')
replace_with_log_transformation(test_data, 'Ram')

# Transformacja logarytmiczna Price_euros
replace_with_log_transformation(train_data, 'Price_euros')
replace_with_log_transformation(val_data, 'Price_euros')
replace_with_log_transformation(test_data, 'Price_euros')

# Transformacja logarytmiczna Inches
replace_with_log_transformation(train_data, 'Inches')
replace_with_log_transformation(val_data, 'Inches')
replace_with_log_transformation(test_data, 'Inches')

# Transformacja logarytmiczna CPU_freq
replace_with_log_transformation(train_data, 'CPU_freq')
replace_with_log_transformation(val_data, 'CPU_freq')
replace_with_log_transformation(test_data, 'CPU_freq')

"""
"""

# One-hot encoding
columns_to_encode = ['OS', 'GPU_company']

train_data_encoded = encode_with_dummies(train_data, columns_to_encode)
val_data_encoded = encode_with_dummies(val_data, columns_to_encode)
test_data_encoded = encode_with_dummies(test_data, columns_to_encode)


# 4. Enkodowanie wartości kategorycznych

In [12]:
data_encoded = train_data_encoded.copy()
label_encoders = {}

# konwersja kolumn kategorycznych na numeryczne korzystając z kodowania etykiet
for col in data_encoded.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data_encoded[col] = le.fit_transform(data_encoded[col])
    label_encoders[col] = le

# pierwsze 20 enkodowanych rekortów
data_encoded.head(20)


,Company,TypeName,Inches,Ram,Price_euros,Touchscreen,CPU_freq,CPU_model,GPU_model,OS_No OS,OS_Other,OS_Windows 10,OS_Windows 7,GPU_company_ARM,GPU_company_Intel,GPU_company_Nvidia
314,1,2,2.533697,1.098612,5.620401,1,0.741937,1,2,False,False,True,False,False,True,False
1032,5,0,2.906901,2.197225,7.084059,0,1.335001,4,1,False,False,True,False,False,False,True
319,0,1,2.533697,1.609438,6.186209,0,0.955511,5,2,False,False,True,False,False,True,False
1039,3,1,2.708050,2.197225,6.919200,0,1.252763,3,2,False,False,True,False,False,True,False
966,2,1,2.809403,1.609438,6.716595,0,1.252763,3,2,False,False,True,False,False,True,False
5,0,1,2.809403,1.609438,5.993961,0,1.386294,5,3,False,False,True,False,False,False,False
100,3,1,2.809403,2.197225,6.579251,0,1.308333,4,4,False,False,True,False,False,False,False
713,1,1,2.708050,1.609438,5.700444,0,0.741937,1,2,False,False,True,False,False,True,False
570,3,2,2.906901,2.197225,7.529401,0,1.335001,3,3,False,False,True,False,False,False,True
548,3,1,2.809403,1.609438,5.986452,0,1.098612,2,2,False,False,True,False,False,True,False


# 5. Zapis przygotowanych danych do pliku

In [13]:
train_data_encoded.to_csv('train_data.csv', index=False)
val_data_encoded.to_csv('val_data.csv', index=False)
test_data_encoded.to_csv('test_data.csv', index=False)